## Задание 1
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [1]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv', index_col='movieId')


def classify_movies(movies, ratings):
    movies['class'] = movies.join(ratings.groupby('movieId').mean()).apply(movie_class, axis=1)

def movie_class(movie):
    try:
        # В условии не сказано, что делать с фильмами с рейтингом 4 и меньше 4.5, поэтому округляем до 0.5        
        rounded = round(movie.rating / 0.5) * 0.5
    except:
        # Для фильма нет оценок
        return 'undefined'
    
    if rounded > 4:
        return 'high'
    elif rounded > 2:
        return 'average'
    else:
        return 'low'
    
classify_movies(movies, ratings)

movies.loc[4705:4725]

,title,genres,class
movieId,,,
4705,"Cage aux Folles, La (1978)",Comedy,average
4706,"Cage aux Folles II, La (1980)",Comedy,average
4709,Paint Your Wagon (1969),Comedy|Musical|Western,average
4710,"Shootist, The (1976)",Drama|Western,average
4711,Theremin: An Electronic Odyssey (1993),Documentary,high
4712,"Wild Child, The (Enfant sauvage, L') (1970)",Drama,undefined
4713,Altered States (1980),Drama|Sci-Fi,average
4714,Any Which Way You Can (1980),Comedy,low
4716,Bad Timing: A Sensual Obsession (1980),Drama,low


## Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:
```
geo_data = {
  'Центр': ['москва', 'тула', 'ярославль'],
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [2]:
import pandas as pd

keywords = pd.read_csv('ml-latest-small/keywords.csv')
geo_data = {
  'Центр': ['москва', 'тула', 'ярославль'],
  'Северо-Запад': ['петербург', 'псков', 'мурманск'],
  'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

def keyword_region(keyword):
    for region in geo_data:
        for town in geo_data[region]:
            if town.lower() in keyword.lower():
                return region
    return 'undefined'

keywords['region'] = keywords['keyword'].apply(keyword_region)
keywords.loc[120:130]

,keyword,shows,region
120,дойки ком,1059292,undefined
121,аск,1045170,undefined
122,мой мир,993332,undefined
123,avito,984448,undefined
124,викинг фильм 2017,984999,undefined
125,школьный портал,982692,undefined
126,вк моя страница,979105,undefined
127,авито москва,979292,Центр
128,спортмастер,968154,undefined
129,новости украины,965720,undefined


## Задание 3 (бонусное)

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл movies.csv и ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

* для каждой строки пройдите по всем годам списка years
* если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
* если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
* Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [3]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')
movies = pd.read_csv('ml-latest-small/movies.csv', index_col='movieId')

def calc_ratings(movies, ratings):
    movies['rating'] = movies.join(ratings.groupby('movieId').mean()).rating

def production_years(movies):
    movies['year'] = movies.apply(production_year, axis = 1)

def production_year(movie):
    for i in range(1950, 2011):
        if str(i) in movie.title:
            return i
    return 1900

def rank_years(movies):
    return movies.groupby('year').mean().sort_values(by='rating', ascending = False)
  
    
calc_ratings(movies, ratings)
production_years(movies)
rank_years(movies)

,rating
year,
1951,3.906653
1952,3.802318
1962,3.772962
1953,3.738878
1957,3.693322
...,...
2002,3.191918
1997,3.183169
1990,3.166759
